In [3]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("../data/checking-logs.sqlite")

schema_test = pd.read_sql("PRAGMA table_info(test_group);", conn)
print("Schema of 'test':\n", schema_test)

test_sample = pd.read_sql("SELECT * FROM test_group LIMIT 10;", conn)
print("First 10 rows of 'test':\n", test_sample)

query_min = """
SELECT
    t.uid,
    MIN((julianday(datetime(dl.deadlines, 'unixepoch')) - julianday(t.first_commit_ts)) * 24) AS min_delta_hours
FROM test_group t
JOIN deadlines dl ON t.labname = dl.labs
WHERE t.labname != 'project1'
GROUP BY t.uid
HAVING min_delta_hours IS NOT NULL
ORDER BY min_delta_hours ASC
LIMIT 1;
"""
df_min = pd.read_sql(query_min, conn)

query_max = """
SELECT
    t.uid,
    MAX((julianday(datetime(dl.deadlines, 'unixepoch')) - julianday(t.first_commit_ts)) * 24) AS max_delta_hours
FROM test_group t
JOIN deadlines dl ON t.labname = dl.labs
WHERE t.labname != 'project1'
GROUP BY t.uid
HAVING max_delta_hours IS NOT NULL
ORDER BY max_delta_hours DESC
LIMIT 1;
"""
df_max = pd.read_sql(query_max, conn)

query_avg = """
SELECT
    AVG((julianday(datetime(dl.deadlines, 'unixepoch')) - julianday(t.first_commit_ts)) * 24) AS avg_delta_hours
FROM test_group t
JOIN deadlines dl ON t.labname = dl.labs
WHERE t.labname != 'project1';
"""
df_avg = pd.read_sql(query_avg, conn)

query_views_diff = """
SELECT
    t.uid,
    AVG((julianday(datetime(dl.deadlines, 'unixepoch')) - julianday(t.first_commit_ts)) * 24) AS avg_diff,
    COUNT(p.datetime) AS pageviews
FROM test_group t
JOIN deadlines dl ON t.labname = dl.labs
LEFT JOIN pageviews p ON t.uid = p.uid
WHERE t.labname != 'project1'
GROUP BY t.uid;
"""
views_diff = pd.read_sql(query_views_diff, conn)

conn.close()

views_diff = views_diff.dropna()
correlation = views_diff[['avg_diff', 'pageviews']].corr().iloc[0, 1]

print("Minimum delta in hours:\n", df_min)
print("Maximum delta in hours:\n", df_max)
print("Average delta in hours:\n", df_avg)
print("Correlation coefficient between pageviews and avg_diff:", correlation)


Schema of 'test':
    cid             name       type  notnull dflt_value  pk
0    0              uid       TEXT        0       None   0
1    1          labname       TEXT        0       None   0
2    2  first_commit_ts  TIMESTAMP        0       None   0
3    3    first_view_ts  TIMESTAMP        0       None   0
First 10 rows of 'test':
        uid   labname             first_commit_ts               first_view_ts
0   user_1    laba04  2020-04-26 17:06:18.462708  2020-04-26 21:53:59.624136
1   user_1   laba04s  2020-04-26 17:12:11.843671  2020-04-26 21:53:59.624136
2   user_1    laba05  2020-05-02 19:15:18.540185  2020-04-26 21:53:59.624136
3   user_1    laba06  2020-05-17 16:26:35.268534  2020-04-26 21:53:59.624136
4   user_1   laba06s  2020-05-20 12:23:37.289724  2020-04-26 21:53:59.624136
5   user_1  project1  2020-05-14 20:56:08.898880  2020-04-26 21:53:59.624136
6  user_10    laba04  2020-04-25 08:24:52.696624  2020-04-18 12:19:50.182714
7  user_10   laba04s  2020-04-25 08:37:54.60